# Packages

In [1]:
### Import packages ###
import itertools
import pandas as pd
import sys
sys.path.append('..')
from utils.Auxiliary import CreateParameterVectorFunction
from utils.Auxiliary import FilterJobNames

---

# Create Parameter Vector

In [2]:
# Data: Iris  MONK1  MONK3  Bar7 (10)  COMPAS (50) | BankNote (10)  BreastCancer (5)  CarEvaluation (10)  FICO (50)  Haberman
Data = "Iris"
ParameterVector = CreateParameterVectorFunction(Data = Data, 
                                  Seed = range(0,50),                       # range(0,50)
                                  RashomonThreshold = 0.025,
                                  DiversityWeight = 0.333,                   
                                  DensityWeight = 0.333,                   
                                  BatchSize = 2,                           # 1 5 or 10
                                  Partition = "short",                                # [short, medium, long, largemem, compute, cpu-g2-mem2x]
                                  Time = "00:59:00",                                     # [00:59:00, 11:59:00, 6-23:59:00]
                                  Memory = "30000M",                                   # [100M, 30000M, 100000M]
                                  IncludeRF = True,
                                  IncludePL = True)
ParameterVector

,Data,Seed,TestProportion,CandidateProportion,SelectorType,ModelType,UniqueErrorsInput,n_estimators,regularization,RashomonThresholdType,RashomonThreshold,Type,DiversityWeight,DensityWeight,BatchSize,Partition,Time,Memory,JobName,Output
0,Iris,0,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,0,100,0.01,Adder,0.025,Classification,0.333,0.333,2,short,00:59:00,30000M,0IS_DA025_B2,Iris/TreeFarms/Raw/0IS_DA025_B2.pkl
1,Iris,0,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0.000,Classification,0.000,0.000,2,short,00:59:00,1000,0IS_PLA0_B2,Iris/RandomForestClassification/Raw/0IS_PLA0_B...
2,Iris,0,0.2,0.8,BatchQBCDiversityFunction,RandomForestClassificationFunction,0,100,0.01,Adder,0.000,Classification,0.333,0.333,2,short,00:59:00,1000,0IS_RFA0_B2,Iris/RandomForestClassification/Raw/0IS_RFA0_B...
3,Iris,0,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,1,100,0.01,Adder,0.025,Classification,0.333,0.333,2,short,00:59:00,30000M,0IS_UA025_B2,Iris/TreeFarms/Raw/0IS_UA025_B2.pkl
4,Iris,1,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,0,100,0.01,Adder,0.025,Classification,0.333,0.333,2,short,00:59:00,30000M,1IS_DA025_B2,Iris/TreeFarms/Raw/1IS_DA025_B2.pkl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Iris,48,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0.000,Classification,0.000,0.000,2,short,00:59:00,1000,48IS_PLA0_B2,Iris/RandomForestClassification/Raw/48IS_PLA0_...
196,Iris,49,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,0,100,0.01,Adder,0.025,Classification,0.333,0.333,2,short,00:59:00,30000M,49IS_DA025_B2,Iris/TreeFarms/Raw/49IS_DA025_B2.pkl
197,Iris,49,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,1,100,0.01,Adder,0.025,Classification,0.333,0.333,2,short,00:59:00,30000M,49IS_UA025_B2,Iris/TreeFarms/Raw/49IS_UA025_B2.pkl
198,Iris,49,0.2,0.8,PassiveLearning,RandomForestClassificationFunction,1,100,0.01,Adder,0.000,Classification,0.000,0.000,2,short,00:59:00,1000,49IS_PLA0_B2,Iris/RandomForestClassification/Raw/49IS_PLA0_...


# Only run these ones

In [3]:
# # ### Simulations that failed ###
# FilterText = ['26B7_UA02']
# ParameterVector = FilterJobNames(ParameterVector,FilterText)
# ParameterVector.index = range(0,len(ParameterVector))
# ParameterVector

---

# Save Parameter Vector

In [4]:
### Resave ###
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
ParameterVector.to_csv(output_path, index=False)